# Convolutional Neural Network

This notebook contains tests of the CNN architecture. Results will later be copied to the main Jupyter Notebook in the root folder of the Project 1.

In [2]:
from tensorflow import keras
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

data_path = os.path.abspath('../data/cinic-10_image_classification_challenge-dataset/train')

seed_value = 2024

train_ds, test_ds = keras.preprocessing.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="both",
    seed=seed_value,
    image_size=(32, 32),
    batch_size=16
)

class_names = train_ds.class_names
print(f'Classes: {class_names}')
input_shape = (32, 32, 3)
n_classes = len(class_names)

def plot_accuracy_and_loss(history_df, name, idx):
    # Plot and save accuraccy
    plt.plot(history_df['accuracy'])
    plt.plot(history_df['val_accuracy'])
    plt.title(f'{name}: accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    path = f'train_history/{name}/{idx}_accuracy.png'
    plt.savefig(path)
    print(f'Accuracy plot is saved to: {path}')
    plt.close()
    # Plot and save loss
    plt.figure()
    plt.plot(history_df['loss'])
    plt.plot(history_df['val_loss'])
    plt.title(f'{name}: loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    path = f'train_history/{name}/{idx}_loss.png'
    plt.savefig(path)
    print(f'Loss plot is saved to: {path}')
    plt.close()

def plot_confusion_matrix(name, idx):
    model = keras.models.load_model('train_history/'+name+'/'+idx+'.keras')
    images, labels = tuple(zip(*test_ds.unbatch()))
    X_test = np.array(images)
    y_test = np.array(labels)
    y_pred = np.argmax(model.predict(X_test), axis=-1)

    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(0, 10), yticklabels=range(0, 10))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{name} Confusion Matrix. Accuracy: {accuracy * 100:.2f}%')
    path = f'train_history/{name}/{idx}_confusion_matrix.png'
    plt.savefig(path)
    print(f'Confusion matrix is saved to: {path}')
    plt.close()
    return accuracy

def plot_accuracy_boxplot(accuracy, name): 
    plt.figure()
    plt.boxplot(accuracy)
    plt.xticks([1], [name])
    plt.title(f'{name}: Accuracy boxplot')
    path = f'train_history/{name}/boxplot.png'
    plt.savefig(path)
    print(f'Accuracy boxplot is saved to: {path}')
    plt.close()

Found 90000 files belonging to 10 classes.
Using 72000 files for training.
Using 18000 files for validation.
Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


## Test 1: Single convolutional layer
Conv2D - MaxPooling2D - (Flatten) Dense (1024) - Dense (10)

In [32]:
name = 'cnn1'
n_epochs = 10
optimizer = 'adamax'
loss = 'sparse_categorical_crossentropy'
n_repeat = 5

keras.utils.set_random_seed(seed_value)

accuracy = []
for idx in range(n_repeat):
  print(f'Attempt #{idx + 1}')
  i = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(1024, activation='relu')(x)
  x = keras.layers.Dense(n_classes, activation='softmax')(x)
  model = keras.Model(i, x)

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=['accuracy']
  )

  history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=n_epochs
  )
  model.save(f'train_history/{name}/{idx}.keras')
  history_df = pd.DataFrame(history.history) 
  hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
  with open(hist_csv_file, mode='w') as f:
      history_df.to_csv(f)
  plot_accuracy_and_loss(history_df, name, str(idx))
  curr_accuracy = plot_confusion_matrix(name, str(idx))
  accuracy.append(curr_accuracy)
  print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')

accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/10
4500/4500 [==============================] - 73s 16ms/step - loss: 6.9584 - accuracy: 0.3302 - val_loss: 1.6841 - val_accuracy: 0.4035
Epoch 2/10
4500/4500 [==============================] - 73s 16ms/step - loss: 1.4700 - accuracy: 0.4743 - val_loss: 1.5687 - val_accuracy: 0.4588
Epoch 3/10
4500/4500 [==============================] - 72s 16ms/step - loss: 1.0609 - accuracy: 0.6315 - val_loss: 1.7221 - val_accuracy: 0.4654
Epoch 4/10
4500/4500 [==============================] - 70s 16ms/step - loss: 0.6399 - accuracy: 0.7858 - val_loss: 2.1110 - val_accuracy: 0.4599
Epoch 5/10
4500/4500 [==============================] - 70s 16ms/step - loss: 0.3356 - accuracy: 0.8956 - val_loss: 2.5846 - val_accuracy: 0.4593
Epoch 6/10
4500/4500 [==============================] - 69s 15ms/step - loss: 0.1860 - accuracy: 0.9459 - val_loss: 2.9754 - val_accuracy: 0.4588
Epoch 7/10
4500/4500 [==============================] - 70s 16ms/step - loss: 0.1157 - accuracy: 0.9682 - val_los

## Test 2: Single convolutional layer with normalization

Conv2D - (Batch normalization) MaxPooling2D - (Flatten) Dense (1024) - Dense (10)

In [34]:
name = 'cnn2'
n_epochs = 10
optimizer = 'adamax'
loss = 'sparse_categorical_crossentropy'
n_repeat = 5

keras.utils.set_random_seed(seed_value)

accuracy = []
for idx in range(n_repeat):
  print(f'Attempt #{idx + 1}')
  i = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(1024, activation='relu')(x)
  x = keras.layers.Dense(n_classes, activation='softmax')(x)
  model = keras.Model(i, x)

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=['accuracy']
  )

  history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=n_epochs
  )
  model.save(f'train_history/{name}/{idx}.keras')
  history_df = pd.DataFrame(history.history) 
  hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
  with open(hist_csv_file, mode='w') as f:
      history_df.to_csv(f)
  plot_accuracy_and_loss(history_df, name, str(idx))
  curr_accuracy = plot_confusion_matrix(name, str(idx))
  accuracy.append(curr_accuracy)
  print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')

accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/10
4500/4500 [==============================] - 77s 17ms/step - loss: 1.6017 - accuracy: 0.4372 - val_loss: 1.4802 - val_accuracy: 0.4733
Epoch 2/10
4500/4500 [==============================] - 77s 17ms/step - loss: 1.2207 - accuracy: 0.5621 - val_loss: 1.3539 - val_accuracy: 0.5294
Epoch 3/10
4500/4500 [==============================] - 77s 17ms/step - loss: 0.9617 - accuracy: 0.6606 - val_loss: 1.4808 - val_accuracy: 0.5253
Epoch 4/10
4500/4500 [==============================] - 77s 17ms/step - loss: 0.6861 - accuracy: 0.7667 - val_loss: 1.6499 - val_accuracy: 0.5269
Epoch 5/10
4500/4500 [==============================] - 78s 17ms/step - loss: 0.4391 - accuracy: 0.8571 - val_loss: 2.0576 - val_accuracy: 0.4988
Epoch 6/10
4500/4500 [==============================] - 77s 17ms/step - loss: 0.2601 - accuracy: 0.9192 - val_loss: 2.1850 - val_accuracy: 0.5096
Epoch 7/10
4500/4500 [==============================] - 78s 17ms/step - loss: 0.1538 - accuracy: 0.9550 - val_los

## Test 3: Dropout layer

Conv2D - (Batch normalization) MaxPooling2D - (Flatten) Dropout - Dense (1024) - Dropout - Dense (10)

In [35]:
name = 'cnn3'
n_epochs = 10
optimizer = 'adamax'
loss = 'sparse_categorical_crossentropy'
n_repeat = 5

keras.utils.set_random_seed(seed_value)

accuracy = []
for idx in range(n_repeat):
  print(f'Attempt #{idx + 1}')
  i = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dropout(0.2)(x)
  x = keras.layers.Dense(1024, activation='relu')(x)
  x = keras.layers.Dropout(0.2)(x)
  x = keras.layers.Dense(n_classes, activation='softmax')(x)
  model = keras.Model(i, x)

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=['accuracy']
  )

  history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=n_epochs
  )
  model.save(f'train_history/{name}/{idx}.keras')
  history_df = pd.DataFrame(history.history) 
  hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
  with open(hist_csv_file, mode='w') as f:
      history_df.to_csv(f)
  plot_accuracy_and_loss(history_df, name, str(idx))
  curr_accuracy = plot_confusion_matrix(name, str(idx))
  accuracy.append(curr_accuracy)
  print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')

accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/10
4500/4500 [==============================] - 78s 17ms/step - loss: 1.6980 - accuracy: 0.4041 - val_loss: 1.5727 - val_accuracy: 0.4527
Epoch 2/10
4500/4500 [==============================] - 78s 17ms/step - loss: 1.3610 - accuracy: 0.5068 - val_loss: 1.3844 - val_accuracy: 0.5098
Epoch 3/10
4500/4500 [==============================] - 78s 17ms/step - loss: 1.1993 - accuracy: 0.5704 - val_loss: 1.4137 - val_accuracy: 0.5074
Epoch 4/10
4500/4500 [==============================] - 78s 17ms/step - loss: 1.0499 - accuracy: 0.6264 - val_loss: 1.4221 - val_accuracy: 0.5316
Epoch 5/10
4500/4500 [==============================] - 78s 17ms/step - loss: 0.9061 - accuracy: 0.6795 - val_loss: 1.4359 - val_accuracy: 0.5421
Epoch 6/10
4500/4500 [==============================] - 78s 17ms/step - loss: 0.7634 - accuracy: 0.7314 - val_loss: 1.5106 - val_accuracy: 0.5420
Epoch 7/10
4500/4500 [==============================] - 78s 17ms/step - loss: 0.6420 - accuracy: 0.7774 - val_los

## Test 4: Multiple convolutional layers

Conv2D - (Batch normalization) Conv2D - (Batch normalization) MaxPooling2D - 

Conv2D - (Batch normalization) Conv2D - (Batch normalization) MaxPooling2D - 

(Flatten) Dropout - Dense (1024) - Dropout - Dense (10)

In [3]:
name = 'cnn4'
n_epochs = 10
optimizer = 'adamax'
loss = 'sparse_categorical_crossentropy'
n_repeat = 5

keras.utils.set_random_seed(seed_value)

accuracy = []
for idx in range(n_repeat):
  print(f'Attempt #{idx + 1}')
  i = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)

  x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)

  x = keras.layers.Flatten()(x)
  x = keras.layers.Dropout(0.2)(x)
  x = keras.layers.Dense(1024, activation='relu')(x)
  x = keras.layers.Dropout(0.2)(x)
  x = keras.layers.Dense(n_classes, activation='softmax')(x)
  model = keras.Model(i, x)

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=['accuracy']
  )

  history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=n_epochs
  )
  model.save(f'train_history/{name}/{idx}.keras')
  history_df = pd.DataFrame(history.history) 
  hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
  with open(hist_csv_file, mode='w') as f:
      history_df.to_csv(f)
  plot_accuracy_and_loss(history_df, name, str(idx))
  curr_accuracy = plot_confusion_matrix(name, str(idx))
  accuracy.append(curr_accuracy)
  print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')

accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/10
4500/4500 [==============================] - 112s 25ms/step - loss: 1.5707 - accuracy: 0.4392 - val_loss: 1.2650 - val_accuracy: 0.5459
Epoch 2/10
4500/4500 [==============================] - 86s 19ms/step - loss: 1.2096 - accuracy: 0.5639 - val_loss: 1.1469 - val_accuracy: 0.5898
Epoch 3/10
4500/4500 [==============================] - 87s 19ms/step - loss: 1.0378 - accuracy: 0.6266 - val_loss: 1.1283 - val_accuracy: 0.6103
Epoch 4/10
4500/4500 [==============================] - 87s 19ms/step - loss: 0.8885 - accuracy: 0.6801 - val_loss: 1.0876 - val_accuracy: 0.6339
Epoch 5/10
4500/4500 [==============================] - 86s 19ms/step - loss: 0.7408 - accuracy: 0.7358 - val_loss: 1.1802 - val_accuracy: 0.6255
Epoch 6/10
4500/4500 [==============================] - 88s 19ms/step - loss: 0.6006 - accuracy: 0.7863 - val_loss: 1.2076 - val_accuracy: 0.6260
Epoch 7/10
4500/4500 [==============================] - 87s 19ms/step - loss: 0.4762 - accuracy: 0.8314 - val_lo